# CraneBrain – Animated Crane Diagrams & Dataset



In [1]:

# === Imports ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

plt.rcParams['figure.dpi'] = 120


## 1. Load Crane Dataset
Update `csv_path` if your file name is different.

In [2]:

# Path to your CSV file in Colab
csv_path = 'Cranes_models_updated (1).csv'  # change if needed

df = pd.read_csv(csv_path)
print("Rows:", len(df))
df.head()


Rows: 35


,Model,Type of Crane,Maximum_Load_Capacity (tons),Maximum_Boom_Length(meters),Radius,Wind Tolerance,Speed
0,XGC88000,Crawler crane,3600.0,120.0,26.34,23.90,8.50
1,XLC 220,Crawler crane,220.0,83.0,29.96,25.90,3.50
2,XCR120,Rough terrain crane,120.0,50.0,44.32,18.97,9.44
3,XCT50Y,Telescopic crane,50.0,43.3,34.31,21.61,3.17
4,XGC45,Crawler crane,45.0,37.0,48.45,22.85,3.28


## 2. Pick a Crane Model
Change `selected_model` to try different cranes from the dataset.

In [3]:

# Show available models
df[['Model', 'Type of Crane']].head(20)


,Model,Type of Crane
0,XGC88000,Crawler crane
1,XLC 220,Crawler crane
2,XCR120,Rough terrain crane
3,XCT50Y,Telescopic crane
4,XGC45,Crawler crane
5,ZTC1100R,Telescopic crane
6,ZCT900V,Crawler crane
7,ZRT600E532,Rough terrain crane
8,ZRT400V432,Rough terrain crane
9,R370-20RB,Tower crane


In [4]:

# Choose a crane model from the 'Model' column
selected_model = 'XGC88000'   # <-- change this string to any model name

row = df[df['Model'] == selected_model].iloc[0]
row


,0
Model,XGC88000
Type of Crane,Crawler crane
Maximum_Load_Capacity (tons),3600.0
Maximum_Boom_Length(meters),120.0
Radius,26.34
Wind Tolerance,23.9
Speed,8.5


## 3. Helper: Crane Geometry from Dataset
We use:
- `Maximum_Boom_Length(meters)` → visual boom length
- `Radius` → how far the hook is projected horizontally

The drawing is schematic (not to physical scale), but it respects the
relative boom length and radius for each crane.

In [5]:

def compute_geometry(row):
    """Return simple geometric parameters for drawing from a row."""
    boom_len = float(row['Maximum_Boom_Length(meters)'])
    radius = float(row['Radius'])

    # Normalize to a reasonable on-screen scale
    max_boom = df['Maximum_Boom_Length(meters)'].max()
    scale = 0.8 / max_boom  # so longest boom ~0.8 of plot height

    boom_len_scaled = boom_len * scale
    radius_scaled = radius * scale

    # Angle: derive a pleasant angle based on radius/boom ratio
    ratio = radius / max(boom_len, 1e-3)
    ratio_clamped = max(0.2, min(ratio, 1.8))
    # Map ratio to angle between 25° and 70°
    angle_deg = 25 + (1.5 - min(ratio_clamped, 1.5)) * 35

    return boom_len_scaled, radius_scaled, np.deg2rad(angle_deg)


## 4. Function to Draw a Single Crane Frame
This draws:
- A base
- A mast
- A boom that can move slightly
- A hanging hook

The time parameter `t` (from 0 to 1) is used for animation.

In [6]:

def draw_crane(ax, boom_len, radius, base_angle_rad, t):
    ax.clear()

    # Background
    ax.set_facecolor('#fdf7e3')

    # Ground line
    ax.plot([-1, 3], [0, 0], color='#444', linewidth=2)

    # Base
    base_width = 0.4
    base_height = 0.1
    base_x = 0.2
    base_y = 0
    ax.add_patch(plt.Rectangle((base_x, base_y), base_width, base_height,
                               color='#555', zorder=3))

    # Mast
    mast_height = 0.35
    mast_top = (base_x + base_width * 0.6, base_y + mast_height)
    ax.plot([base_x + base_width * 0.6, mast_top[0]],
            [base_y + base_height, mast_top[1]],
            color='#c58b00', linewidth=6, solid_capstyle='round')

    # Animated boom angle around base_angle_rad
    delta = np.deg2rad(8)  # +/- 8 degrees
    angle = base_angle_rad + (t - 0.5) * 2 * delta

    boom_dx = boom_len * np.cos(angle)
    boom_dy = boom_len * np.sin(angle)
    boom_end = (mast_top[0] + boom_dx, mast_top[1] + boom_dy)

    ax.plot([mast_top[0], boom_end[0]],
            [mast_top[1], boom_end[1]],
            color='#f0a500', linewidth=6, solid_capstyle='round')

    # Hook
    hook_drop = 0.35
    hook_y = max(boom_end[1] - hook_drop, 0.05)
    ax.plot([boom_end[0], boom_end[0]],
            [boom_end[1], hook_y],
            color='black', linewidth=2)
    ax.scatter([boom_end[0]], [hook_y], s=80, color='red', zorder=5)

    # Labels
    ax.text(0.05, 0.9, f"Model: {row['Model']}",
            transform=ax.transAxes, fontsize=11, weight='bold')
    ax.text(0.05, 0.85,
            f"Type: {row['Type of Crane']}",
            transform=ax.transAxes, fontsize=9)
    ax.text(0.05, 0.8,
            f"Boom: {row['Maximum_Boom_Length(meters)']} m  |  Radius: {row['Radius']} m",
            transform=ax.transAxes, fontsize=9)

    ax.set_xlim(-0.2, 2.2)
    ax.set_ylim(-0.1, 1.4)
    ax.axis('off')


## 5. Create Animation for Selected Crane
This will create a looped animation where the boom gently moves
around its base angle, giving a dynamic engineering-style visualization.

In [7]:

boom_len_scaled, radius_scaled, base_angle_rad = compute_geometry(row)

fig, ax = plt.subplots(figsize=(5, 6))

def update(frame):
    t = frame / 60  # 60 frames
    draw_crane(ax, boom_len_scaled, radius_scaled, base_angle_rad, t)
    return []

anim = animation.FuncAnimation(fig, update, frames=60, interval=80, blit=False)

plt.close(fig)
HTML(anim.to_jshtml())


### Tips
- Change `selected_model` to any other crane in the dataset and rerun the animation cell.
- Tweak colors, line widths, and geometry in `draw_crane` to better match your UI designs.
- This is a schematic visual; it is meant for presentations, not for structural design.


In [8]:
# =================================
# 3. Risk Assessment Engine
# =================================

def compute_risk(crane_row, load_weight, lift_radius, wind_speed):
    """
    Simple engineering-style risk model.
    crane_row: one row from df (Series)
    load_weight: tons
    lift_radius: meters
    wind_speed: m/s (or same unit as 'Wind Tolerance')
    """
    capacity = float(crane_row["Maximum_Load_Capacity (tons)"])
    design_radius = float(crane_row["Radius"])
    wind_tol = float(crane_row["Wind Tolerance"])

    # ---- Load utilization (0–100%) ----
    load_util = (load_weight / capacity) * 100
    load_util = max(0, load_util)

    # ---- Radius utilization (0–100%) ----
    radius_util = (lift_radius / design_radius) * 100
    radius_util = max(0, radius_util)

    # ---- Wind compliance (0–100% good) ----
    wind_ratio = wind_speed / wind_tol
    wind_compliance = max(0, 1 - wind_ratio) * 100  # 100 is perfect, 0 is bad

    # ---- Stability proxy (0–100% good) ----
    # Higher radius = lower stability
    stability = max(0, 1 - (lift_radius / (design_radius * 1.2))) * 100

    # ---- Overall risk score (0=best, 100=worst) ----
    # You can tune weights as you like
    risk_score = (
        (load_util) * 0.35 +           # high load → higher risk
        (radius_util) * 0.25 +         # large radius → higher risk
        (100 - wind_compliance) * 0.25 +  # low wind compliance → higher risk
        (100 - stability) * 0.15       # low stability → higher risk
    )

    # Clamp between 0–100
    risk_score = max(0, min(100, risk_score))

    # ---- Status bucket ----
    if risk_score < 30:
        status = "Optimal"
    elif risk_score < 60:
        status = "Moderate"
    else:
        status = "High Risk"

    return {
        "load_util_percent": round(load_util, 2),
        "radius_util_percent": round(radius_util, 2),
        "wind_compliance_percent": round(wind_compliance, 2),
        "stability_percent": round(stability, 2),
        "risk_score": round(risk_score, 1),
        "status": status,
    }


In [9]:
# =================================
# 4. HSE Compliance Checker
# =================================

def hse_compliance_check(crane_row, load_weight, lift_radius, wind_speed):
    """
    Very simple rule-based HSE checks.
    You can extend this with real HSE 2025 rules.
    """
    capacity = float(crane_row["Maximum_Load_Capacity (tons)"])
    design_radius = float(crane_row["Radius"])
    wind_tol = float(crane_row["Wind Tolerance"])
    boom_len = float(crane_row["Maximum_Boom_Length(meters)"])

    issues = []

    # Load within 75% of capacity (safety margin)
    if load_weight > 0.75 * capacity:
        issues.append("Load > 75% of rated capacity (consider reducing load).")

    # Radius within 95% of design radius
    if lift_radius > 0.95 * design_radius:
        issues.append("Lift radius very close to maximum (risk of instability).")

    # Wind limit
    if wind_speed > wind_tol:
        issues.append("Wind speed exceeds crane's rated wind tolerance.")
    elif wind_speed > 0.8 * wind_tol:
        issues.append("Wind speed is >80% of tolerance, monitor conditions closely.")

    # Long boom check
    if boom_len > 80:
        issues.append("Long boom length – verify ground conditions and boom deflection.")

    if not issues:
        status = "Fully HSE Compliant"
    else:
        status = "HSE Warnings Present"

    return {
        "hse_status": status,
        "hse_issues": issues,
    }


In [10]:
# =================================
# 5. Recommendation Engine
# =================================

def generate_recommendations(risk_result, hse_result):
    recs = []

    # Based on risk level
    if risk_result["status"] == "High Risk":
        recs.append("❗ High risk: reconfigure lift before proceeding.")
        recs.append("• Reduce load weight or split into multiple lifts.")
        recs.append("• Decrease operating radius if possible.")
        recs.append("• Use a higher capacity crane for this lift.")
    elif risk_result["status"] == "Moderate":
        recs.append("⚠ Moderate risk: proceed with enhanced monitoring.")
        recs.append("• Verify outrigger setup and ground stability.")
        recs.append("• Monitor wind and weather throughout the lift.")
    else:
        recs.append("✅ Configuration is within safe limits.")
        recs.append("• Maintain standard HSE procedures and supervision.")

    # Add HSE issues
    for issue in hse_result["hse_issues"]:
        recs.append("HSE: " + issue)

    return recs


In [12]:
# Choose a crane model from the 'Model' column
selected_model = 'XGC88000'   # change this string to any model name

row = df[df['Model'] == selected_model]

if row.empty:
    raise ValueError(f"Model '{selected_model}' not found in dataset")

crane = row.iloc[0]   # <-- THIS creates the `crane` variable
crane


,0
Model,XGC88000
Type of Crane,Crawler crane
Maximum_Load_Capacity (tons),3600.0
Maximum_Boom_Length(meters),120.0
Radius,26.34
Wind Tolerance,23.9
Speed,8.5


In [13]:
# =================================
# 6. Run Full Analysis for One Scenario
# =================================

# Example lift scenario – change these:
lift_load = 200      # tons
lift_radius = 40     # meters
wind_speed = 15      # same unit as 'Wind Tolerance' in dataset

print(f"Model: {crane['Model']}")
print(f"Type:  {crane['Type of Crane']}")
print("--------------------------------------------------")

risk = compute_risk(crane, load_weight=lift_load,
                    lift_radius=lift_radius,
                    wind_speed=wind_speed)

hse = hse_compliance_check(crane, load_weight=lift_load,
                           lift_radius=lift_radius,
                           wind_speed=wind_speed)

recs = generate_recommendations(risk, hse)

print("RISK RESULT:")
for k, v in risk.items():
    print(f"  {k}: {v}")

print("\nHSE COMPLIANCE:")
print("  Status:", hse["hse_status"])
if hse["hse_issues"]:
    for issue in hse["hse_issues"]:
        print("  -", issue)
else:
    print("  - No issues detected.")

print("\nRECOMMENDATIONS:")
for r in recs:
    print(" ", r)


Model: XGC88000
Type:  Crawler crane
--------------------------------------------------
RISK RESULT:
  load_util_percent: 5.56
  radius_util_percent: 151.86
  wind_compliance_percent: 37.24
  stability_percent: 0
  risk_score: 70.6
  status: High Risk

HSE COMPLIANCE:
  Status: HSE Warnings Present
  - Lift radius very close to maximum (risk of instability).
  - Long boom length – verify ground conditions and boom deflection.

RECOMMENDATIONS:
  ❗ High risk: reconfigure lift before proceeding.
  • Reduce load weight or split into multiple lifts.
  • Decrease operating radius if possible.
  • Use a higher capacity crane for this lift.
  HSE: Lift radius very close to maximum (risk of instability).
  HSE: Long boom length – verify ground conditions and boom deflection.


In [15]:
# ===============================================================
# CRANEBRAIN – FULL XGBOOST ML PIPELINE IN ONE SINGLE CELL
# ===============================================================

# 1) Install dependencies
!pip install xgboost --quiet

# 2) Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from xgboost import XGBClassifier

# 3) Load dataset (upload file in Colab first)
df = pd.read_csv("Cranes_models_updated (1).csv")
print("Loaded:", len(df), "rows")

# 4) Clean column names
# First, strip any whitespace from column names
df.columns = df.columns.str.strip()

# Then, rename specific columns to match the format used in generate_risk and features list
df.rename(columns={
    "Maximum_Load_Capacity (tons)": "Maximum_Load_Capacity_tons",
    "Maximum_Boom_Length(meters)": "Maximum_Boom_Length_meters",
    "Wind Tolerance": "Wind_Tolerance"
}, inplace=True)

# For any other columns, replace spaces with underscores (if any remain)
df.columns = df.columns.str.replace(" ", "_")

# 5) Synthetic risk label generator (for supervised ML)
def generate_risk(row):
    score = 0

    # Low capacity → more risk
    if row.Maximum_Load_Capacity_tons < 80:
        score += 2

    # Long boom → more risk
    if row.Maximum_Boom_Length_meters > 60:
        score += 1

    # Large radius → more instability
    if row.Radius > 40:
        score += 2

    # Weak wind tolerance → dangerous
    if row.Wind_Tolerance < 20:
        score += 2

    # Convert score → class
    if score <= 1:
        return "Safe"
    elif score <= 3:
        return "Moderate"
    else:
        return "High_Risk"

# Generate ML target
df["Risk_Class"] = df.apply(generate_risk, axis=1)

# Encode labels
le = LabelEncoder()
df["Risk_Label"] = le.fit_transform(df["Risk_Class"])

# 6) Feature selection
features = [
    "Maximum_Load_Capacity_tons",
    "Maximum_Boom_Length_meters",
    "Radius",
    "Wind_Tolerance",
    "Speed"
]

X = df[features]
y = df["Risk_Label"]

# 7) Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

# 8) Train XGBoost model
model = XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.08,
    subsample=0.9,
    colsample_bytree=0.9,
    eval_metric="mlogloss"
)

model.fit(X_train, y_train)

# 9) Evaluate
pred = model.predict(X_test)

print("\n Accuracy:", accuracy_score(y_test, pred))
print("\n Classification Report:\n")
print(classification_report(y_test, pred, target_names=le.classes_))

# 10) ML Prediction for new lift scenario
def predict_lift_risk(crane_model, load, radius, wind):
    row = df[df.Model == crane_model].iloc[0]

    # match ML feature structure
    feat = pd.DataFrame([{
        "Maximum_Load_Capacity_tons": row.Maximum_Load_Capacity_tons,
        "Maximum_Boom_Length_meters": row.Maximum_Boom_Length_meters,
        "Radius": radius,
        "Wind_Tolerance": row.Wind_Tolerance - wind,  # wind margin
        "Speed": row.Speed
    }])

    pred_label = model.predict(feat)[0]
    risk_class = le.inverse_transform([pred_label])[0]
    return risk_class

print("\n Example Prediction:")
print(predict_lift_risk("XGC88000", load=150, radius=30, wind=12))

# 11) Automatic safest crane selector
def find_safest_crane(load, radius, wind):
    results = []
    for _, row in df.iterrows():
        risk = predict_lift_risk(row.Model, load, radius, wind)
        results.append((row.Model, risk))
    return pd.DataFrame(results, columns=["Model", "Predicted_Risk"]).sort_values("Predicted_Risk")

print("\n✅ Safest crane for scenario:")
find_safest_crane(load=200, radius=40, wind=15).head(10)

Loaded: 35 rows

 Accuracy: 0.7777777777777778

 Classification Report:

              precision    recall  f1-score   support

   High_Risk       1.00      0.80      0.89         5
    Moderate       0.50      1.00      0.67         2
        Safe       1.00      0.50      0.67         2

    accuracy                           0.78         9
   macro avg       0.83      0.77      0.74         9
weighted avg       0.89      0.78      0.79         9


 Example Prediction:
High_Risk

✅ Safest crane for scenario:


,Model,Predicted_Risk
0,XGC88000,High_Risk
19,CXT Explorer,High_Risk
20,SCE1350A-EV,High_Risk
21,STC600C5,High_Risk
22,SFT1100 (T80108-50),High_Risk
23,SRC500T,High_Risk
24,SPS8000,High_Risk
18,CXT Gantry crane,High_Risk
25,AC 9.700-1,High_Risk
27,AC 3.055-1,High_Risk
